In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import bokeh as bk
from collections import Counter


In [2]:
### Import data

data_dir = '/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs'
filename = 'Vahid_std.csv'
filepath = os.path.join(data_dir,filename)

df = pd.read_csv(filepath)
#print(df)

In [3]:
hyst_col = {'n_cycles', 'hysteresis_area', 'stiffness', 'creep'}
hyst_df = pd.DataFrame(columns = hyst_col)
#hyst_df.n_cycles = n_cycles


In [4]:
### Extract number of cycles without repeating values in other table, and store number of measurements per cycle

n_cycles = sorted(set(df.Machine_N_cycles))
n_measurements = Counter(df.Machine_N_cycles)  


#print(n_cycles)
#print(n_measurements)
n_cycles_df = pd.DataFrame(n_cycles)
#n_cycles_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_n_cycles.csv', index=False)

hyst_df.n_cycles = n_cycles



In [5]:
### Isolate Stress/Strain values for each cycles


Stress_N = []
Strain_N = []
df0=np.array(df)
for k in range(len(n_cycles)):
    Stress_N.append([df0[i,14] for i in range(df0.shape[0]) if df0[i,1]==n_cycles[k]])
    Strain_N.append([df0[i,0] for i in range(df0.shape[0]) if df0[i,1]==n_cycles[k]])
    
    
print(Stress_N)
#print(Strain_N)
#print(len(Stress_N))

[[], [30.0611232941886, 38.6930641219748, 42.299559834432394, 44.9009665361558, 46.7928986608041, 47.5614960811183, 45.492195327108, 42.890788633354894, 38.8704327651351, 28.878665617134104, 23.7349746402861, 18.2365462089929, 12.797240517514998, 7.9491635840069295, 6.82582866856017, 4.10617569025306, 4.1652985814629, 4.75652749264086, 6.70758288763333, 10.254956286102301, 14.2161898422966, 19.1233895144796, 23.9714661823521], [28.8195427330966, 33.6084962829699, 39.343415812772996, 43.1864030321383, 45.96517835856171, 47.50237320274201, 46.142546995056804, 44.9009665361558, 41.2353479918224, 37.1558692078311, 32.6625301588598, 26.927610434619, 22.0795338380308, 12.1468887403769, 8.30390092446308, 5.288633511269019, 4.87477327467552, 5.288633511269019, 6.648459997144692, 9.7819731698204, 13.743206735127801, 17.349702899632, 22.0204109519841, 27.3414706264466], [30.8297207804931, 35.3821827534183, 40.1120132627331, 43.83675470786961, 46.2016698738521, 47.2067588105869, 47.91623335099309

In [6]:
### Definition of polyarea and stiffness
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))


In [7]:
### Hysteresis loops area computation

Hysteresis_Area = []
Stiffness = []

for j in range(len(n_cycles)):
    x=Stress_N[j]
    y=Strain_N[j]
    points=(x, y)
    Hysteresis_Area.append(PolyArea(x, y))
    #Stress_min = np.min


hyst_df.hysteresis_area = Hysteresis_Area
#hyst_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_hystarea.csv', index=False)



In [8]:
### Stiffness computation


Stress_max = [np.max(Stress_N[i]) for i in range(1,len(n_cycles))]
Stress_min = [np.min(Stress_N[i]) for i in range(1,len(n_cycles))]
Strain_max = [np.max(Strain_N[i]) for i in range(1,len(n_cycles))]
Strain_min = [np.min(Strain_N[i]) for i in range(1,len(n_cycles))]
creep = []
                    
Stiffness = []

for i in range(len(n_cycles)-1):
    dStress = Stress_max[i]-Stress_min[i]
    dStrain = Strain_max[i]-Strain_min[i]
    Stiffness.append(dStress/dStrain)
    creep.append(Strain_min[i])
    

Stiffness.append(6331.946706)
creep.append(0.008375)
hyst_df.stiffness = Stiffness
hyst_df.creep = creep
#stif_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_stiff.csv', index=False)

In [9]:
print(hyst_df)

       n_cycles     creep     stiffness  hysteresis_area
0           NaN  0.000570  43687.822473         0.000000
1           1.0  0.000868  29217.481622         0.000401
2        1000.0  0.000986  27391.381050         0.001224
3        2000.0  0.001104  25264.809740         0.001222
4        3000.0  0.001283  23663.124346         0.003683
...         ...       ...           ...              ...
1194  1193000.0  0.007110   6937.089743         0.032850
1195  1194000.0  0.007252   6698.079021         0.035368
1196  1195000.0  0.007650   6566.353163         0.033975
1197  1196000.0  0.008375   6331.946706         0.033212
1198  1197000.0  0.008375   6331.946706         0.032153

[1199 rows x 4 columns]


In [10]:
hyst_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_hysteresis_analysis.csv', index=False)
